In [1]:
import os
from importlib import reload
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
import math
import datetime
import pdblp
import runpy
import QuantLib as ql
import matplotlib as mpl
#import matplotlib
mpl.get_backend()
import matplotlib.pyplot as plt
plt.interactive(False)
import seaborn as sns
from tabulate import tabulate
from sklearn.preprocessing import minmax_scale
from sklearn.mixture import GaussianMixture
from scipy.stats import zscore
import pickle
import re
import concurrent.futures
import time
from datetime import datetime, timedelta, time
from scipy import stats

In [2]:
pd.set_option('display.max_columns', 10000)
pd.set_option('display.width', 10000)
pd.options.display.float_format = '{:,}'.format
pd.options.mode.chained_assignment = None

## BBG API
con = pdblp.BCon(debug=False, port=8194, timeout=50000)
con.start()

from Utilities import *
from Conventions import FUT_CT,FUT_CT_Q, ccy, ccy_infl
from OIS_DC_BUILD import ois_dc_build, get_wirp
from SWAP_BUILD import swap_build
from SWAP_PRICER import Swap_Pricer, Swap_curve_fwd
from SWAP_TABLE import swap_table, swap_table2, curve_hmap
from INF_ZC_BUILD import infl_zc_swap_build, Infl_ZC_Pricer, inf_swap_table
from BOND_CURVES import bond_curve_build
from VOL_BUILD import build_vol_surf, build_vol_spline, bond_fut_opt_strat, get_sim_option_px, build_stir_vol_surf, stir_opt_strat
from PLOT import plt_curve, plt_inf_curve, plt_opt_strat, rates_hm, curve_hm, plt_ois_curve, plot_opt_vol_surf, plt_stir_opt_strat, plotool, ecfc_plot


In [3]:
sofr = ois_dc_build('SOFR_DC', b=0)
sofr.rates

*** !!! ois_curve built !!! ***


,Tenor,SwapRate
0,2Y,4.4615
1,3Y,4.2015
2,4Y,4.0671
3,5Y,3.9902
4,6Y,3.9502
5,7Y,3.9257
6,8Y,3.9115
7,9Y,3.9047
8,10Y,3.9014
9,12Y,3.908


In [3]:
# bokeh imports
from bokeh.plotting import figure, show
from bokeh.models import HoverTool, BoxZoomTool, ResetTool, Legend, DatetimeTickFormatter, SingleIntervalTicker, LinearAxis, FuncTickFormatter
from bokeh.models import ColumnDataSource, TabPanel, Tabs, LabelSet, Span, Range1d, FactorRange, Range, Label
from bokeh.layouts import row, column, gridplot
from bokeh.palettes import Category10, brewer, Category20, Bright6
from bokeh.transform import factor_cmap
from bokeh.models.renderers import GlyphRenderer
from itertools import accumulate

In [20]:
#p = figure(title="Simple Line Example", x_axis_label='x', y_axis_label='y')
p = figure(tools=["pan","hover","wheel_zoom","box_zoom","save","reset","help"])
p.line([1, 2, 3, 4, 5], [6, 7, 2, 4, 5], legend_label="Temp.", line_width=2,  color="firebrick", alpha=0.8)
p.title.text = "Customized Line Plot"
p.title.text_color = "olive"
p.title.text_font = "times"
p.title.text_font_style = "italic"

show(p)

In [12]:
## Scatter

p = figure(title="Simple Scatter Example")
p.scatter(x=[1, 2, 3, 4, 5], y=[6, 7, 2, 4, 5], size=15, line_color="navy", fill_color="orange", alpha=0.5)


show(p)

In [16]:
## Bar plot

source = ColumnDataSource(data=dict(x=["A", "B", "C", "D"], counts=[50, 40, 65, 10]))
p = figure(x_range=["A", "B", "C", "D"], title="Bar Example", toolbar_location=None, tools="")
p.vbar(x='x', top='counts', width=0.9, source=source)
show(p)

In [25]:
## Layout
t=["pan","hover","wheel_zoom","box_zoom","save","reset","help"]
s1 = figure(min_width=250, min_height=250, tools = t)
s1.circle([1, 2, 3, 4, 5], [6, 7, 2, 4, 5])

s2 = figure(min_width=250, min_height=250, tools = t)
s2.line([1, 2, 3, 4, 5], [6, 7, 2, 4, 5])

layout = row(s1, s2)
show(layout)

In [74]:
p = figure(title="Fill Below Line with VArea")
x = [1, 2, 3, 4, 5]
y = [6, 7, 2, 4, 5]

p.line(x, y, line_color="blue")
p.varea(x=x, y1=[0]*len(x), y2=y, color="blue", alpha=0.3)
show(p)

In [10]:
sofr=ois_dc_build('SOFR_DC', b=0)
swtb1 = swap_table(sofr, offset = [-1])



*** !!! ois_curve built !!! ***
*** !!! ois_curve built !!! ***


In [8]:
swtb1.all_curves[0].ref_date

Date(26,1,2024)

In [4]:
#built_curve = swtb1.all_curves
#built_curve ==0

swtb1.all_curves[1].curve.discount(ql.Date(25,1,2054))

0.3434206113045937

In [5]:
cur_1 = ois_dc_build('SOFR_DC', b=-1)

print(cur_1.curve.discount(ql.Date(25,1,2054)))

print(100*cur_1.curve.forwardRate(ql.Date(24,1,2026), ql.Date(24,1,2026), ql.Actual365Fixed(), ql.Simple).rate())
print(100*swtb1.all_curves[1].curve.forwardRate(ql.Date(24,1,2026), ql.Date(24,1,2026), ql.Actual365Fixed(), ql.Simple).rate())

*** !!! ois_curve built !!! ***
0.34338845486062636
3.295973219685777
3.2959732199078218


In [6]:
#def plt_ois_curve(c1, h1=[0], max_tenor=30, bar_chg = 0, sprd = 0, name = '',fwd_tenor = '1y',int_tenor = '1y', tail = 1, curve_fill = "", label_curve_name = 1):
#    mpl.rcParams.update(mpl.rcParamsDefault)
    #### build curves
c1 = ['SOFR_DC']
#    h1 = [0,'15-03-2019']
h1 = [0, -1]
bar_chg = 1
sprd = 0
max_tenor = 30
tail = 1
fwd_tenor = '1y'
int_tenor = '1y'
name = 'Fwd Tenors: '+fwd_tenor
curve_fill = ''
label_curve_name = 1
built_curve = swtb1.all_curves
#built_curve = 0

n_ccy = len(c1)
n_chg = len(h1)

today = ql.Date(datetime.datetime.now().day,datetime.datetime.now().month,datetime.datetime.now().year)
if built_curve == 0:
    crv_list = {}
    for k in np.arange(n_ccy):
        c2 = ccy(c1[k], today)
        if c2.ois_trigger == 0:
            crv_list[c1[k]] = [swap_build(c1[k], i) for i in h1]
        else:
            crv_list[c1[k]] = [ois_dc_build(c1[k], b=h1[i]) for i in np.arange(len(h1))]

    crv = flat_lst(list(crv_list.values()))
else:
    crv = built_curve

h2 = [crv[i].trade_date for i in np.arange(len(crv))]

if fwd_tenor[-1] == 'd':
    fwd_tenor2 = ql.Days
elif fwd_tenor[-1] == 'm':
    fwd_tenor2 = ql.Months
else:
    fwd_tenor2 = ql.Years

if ((bar_chg == 0 ) & (sprd == 0)):
    n_plots = 1
    n_obj = {'curve': [n_ccy*n_chg]}
elif ((bar_chg == 1 ) & (sprd == 0)):
    if (n_chg < 3):
        n_plots = 2
        n_obj = {'curve':[n_ccy*n_chg], 'chg':[n_ccy]}

    elif ((n_ccy == 1) & (n_chg > 3)):
        n_plots = 2
        n_obj = {'curve': [n_ccy*n_chg], 'chg': [n_chg-1]}
    else:
        n_plots = 1 + n_ccy
        n_obj = {'curve': [n_ccy*n_chg], 'chg' : [n_chg-1]*n_ccy }

elif ((bar_chg == 0 ) & (sprd == 1)):
    n_plots = 2
    n_obj = {'curve ': [n_ccy*n_chg], 'chg': [n_ccy-1] }

else:
    if (n_chg < 3):
        n_plots = 3
        n_obj = {'curve': [n_ccy*n_chg], 'sprd': [n_ccy-1], 'chg': [n_ccy-1] }
    elif ((n_ccy == 2) & (n_chg > 3)):
        n_plots = 3
        n_obj = { 'curve': [n_ccy*n_chg], 'sprd': [n_ccy-1], 'chg': [n_chg-1] }
    else:
        n_plots = 1 + n_ccy
        n_obj = { 'curve': [n_ccy*n_chg], 'sprd' : [n_ccy-1],  'chg' : [n_chg-1]*(n_ccy-1) }

#### Set Layout?



#### Get ALL Data
rates = dict([(key, []) for key in c1])
for i in np.arange(len(crv)):
    if crv[i].ois_trigger == 1:
        i=1
        d2 = crv[i].ref_date
        print('d2:', d2)
        d3 = d2 + ql.Period(max_tenor,ql.Years)
#       dates_in = [ ql.Date(serial) for serial in range(d2.serialNumber(),d3.serialNumber()+1) ]   #### dates for plotting !!
#       yr_axis = [(dates_in[i]-dates_in[0])/365.25 for i in range(len(dates_in)) ]
        dates_in2 = ql.MakeSchedule(d2, d3, ql.Period(int_tenor))  #### dates for pricing !!
        yr_axis = [(dates_in2[i]-dates_in2[0])/365.25 for i in range(len(dates_in2)) ]
        for d in dates_in2:
            print(d)
            print(crv[i].ref_date)
            print(crv[i].cal.advance(d,int(fwd_tenor[0]),fwd_tenor2))
            print(100*crv[i].curve.forwardRate(d, crv[i].cal.advance(d,int(fwd_tenor[0]),fwd_tenor2), ql.Actual365Fixed(), ql.Simple).rate())


        rates_c = [100*crv[i].curve.forwardRate(d, crv[i].cal.advance(d,int(fwd_tenor[0]),fwd_tenor2), ql.Actual365Fixed(), ql.Simple).rate() for d in dates_in2]
        rates[c1[int(np.floor(i/len(h1)))]].append(rates_c)

    else:
        yr_axis = np.arange(max_tenor)
        rates_c = []
        j = 0
        while j < max_tenor:
            rates_c.append(Swap_Pricer([[crv[i],j,tail]]).rate[0])
            j+=1
        rates[c1[int(np.floor(i/len(h1)))]].append(rates_c)

rates_change = dict([(key, []) for key in c1])
for j in rates.keys():
    k = 1
    while k < n_chg:
        rates_diff = 100*( np.array(rates[j][0]) - np.array(rates[j][k]))
        rates_change[j].append(rates_diff.tolist())
        k+=1
bar_dict = rates_change

if sprd == 1:
    c2 = [c1[0]+' - '+c1[i] for i in np.arange(1,n_ccy)]
    spreads = dict([(key, []) for key in c2])
    for i,j in enumerate(spreads.keys()):
        spreads[j] = 100*(np.array(rates[list(rates.keys())[0]]) - np.array(rates[list(rates.keys())[i+1]]))

    spreads_change = dict([(key, []) for key in c2])
    for j in spreads.keys():
        k = 1
        while k < n_chg:
            sprd_chg = 1*( np.array(spreads[j][0]) - np.array(spreads[j][k]))
            spreads_change[j].append(sprd_chg.tolist())
            k+=1
    bar_dict = spreads_change


## Plot curve
s_plot = []
yr_axis2 = np.array(yr_axis)
s1 = figure(width=700, height=400, tools = ["pan","hover","wheel_zoom","box_zoom","save","reset","help"])
s1.xgrid.visible = False
s1.ygrid.visible = False

for i in rates.keys():
    [s1.line(yr_axis2, rates[i][j],
             legend_label = [str(ccy(i,today).curncy)+': '+str(h2[j]), i+': '+str(h2[j])][label_curve_name],
             line_width = 2,
             color =  Category20[20][(j+2*(list(rates.keys()).index(i))+1)]) for j in np.arange(n_chg)]
#   [s1.circle(yr_axis2, rates[i][j], size=0.5, fill_color= Category20[20][2*list(rates.keys()).index(i)]) for j in np.arange(n_chg)]
#   print(Category20[20][2*list(rates.keys()).index(i)])
    if len(curve_fill) > 0:
        for j in np.arange(len(curve_fill)):
            x_0 = np.where([(yr_axis2 > curve_fill[j][0])])[1][0]
            x_1 = np.where([(yr_axis2 < curve_fill[j][1])])[1][-1]
            s1.varea(x=yr_axis2[x_0:x_1+1], y1=[0]*len(yr_axis2[x_0:x_1+1]), y2=rates[i][0][x_0:x_1+1], color="green", alpha=0.7)
s1.legend.label_text_font = "calibri"
s1.legend.label_text_font_size = "9pt"
s1.legend.spacing = 1
s1.xaxis.axis_label = 'Rate'
s1.yaxis.axis_label = 'Tenor'
s_plot.append(s1)
#layout = row(s1)

## Plot Sprd
if sprd == 1:
    s2 = figure(width=700, height=300, tools = ["pan","hover","wheel_zoom","box_zoom","save","reset","help"])
    s2.xgrid.grid_line_dash = 'dotted'
    s2.ygrid.grid_line_dash = 'dotted'
#    plt.setp(axs[1], xlim=axs[0].get_xlim())
    for j in spreads.keys():
        s2.line( yr_axis , spreads[j][0],
                 line_width = 2,
                 legend_label = str(j),
                 color = Category20[20][(2*(list(bar_dict.keys()).index(j))+1)] )
    s2.legend.label_text_font = "calibri"
    s2.legend.label_text_font_size = "9pt"
    s2.legend.spacing = 1
    s2.xaxis.axis_label = 'Spread'
    s2.yaxis.axis_label = 'Tenor'
    s_plot.append(s2)


## Plot Chg
if sprd == 1:
    start_sub_chg = 2
else:
    start_sub_chg = 1

if bar_chg == 1:
    n_sub_chg = len(n_obj['chg'])
    if n_sub_chg == 1:
        s3 = figure(width=700, height=300, tools = ["pan","hover","wheel_zoom","box_zoom","save","reset","help"])
        s3.xgrid.grid_line_dash = 'dotted'
        s3.ygrid.grid_line_dash = 'dotted'
        width = 0.15
        bar_yr_axis = yr_axis
        for i in bar_dict.keys():
            for j in np.arange(len(bar_dict[i])):
                s3.vbar(x=np.array(bar_yr_axis), top=bar_dict[i][j],
                        legend_label=i+': '+str(h1[j+1]),
                        width=width,
                        color = Category20[20][(j+2*(list(bar_dict.keys()).index(i))+1)] )
                bar_yr_axis = np.array(bar_yr_axis)+width+0.1
        s3.legend.label_text_font = "calibri"
        s3.legend.label_text_font_size = "9pt"
        s3.legend.spacing = 1
        s3.xaxis.axis_label = 'Chg'
        s3.yaxis.axis_label = 'Tenor'
        s_plot.append(s3)
    else:
        for j in np.arange(n_sub_chg):
            s4 = figure(width=700, height=300, tools = ["pan","hover","wheel_zoom","box_zoom","save","reset","help"])
            s4.xgrid.grid_line_dash = 'dotted'
            s4.ygrid.grid_line_dash = 'dotted'
            width = 0.15
            bar_yr_axis = yr_axis
            for i in np.arange(len(bar_dict[ list(bar_dict.keys())[j]  ])):
                s4.vbar(x=np.array(bar_yr_axis)+1, top=bar_dict[list(bar_dict.keys())[j]][i],
                        legend_label= list(bar_dict.keys())[j]+': '+str(h1[i+1]),
                        width=width,
                        color = Category20[20][(j+(2*i)+1)] )
                bar_yr_axis = np.array(bar_yr_axis)+width+0.1
                s4.legend.label_text_font = "calibri"
                s4.legend.label_text_font_size = "9pt"
                s4.legend.spacing = 1
                s_plot.append(s4)




layout = column(*s_plot)
show(layout)


d2: January 25th, 2024
January 25th, 2024
January 25th, 2024
January 27th, 2025
4.8869444444424115
January 25th, 2025
January 25th, 2024
January 26th, 2026
3.6132023151837327
January 25th, 2026
January 25th, 2024
January 25th, 2027
3.4299941006026735
January 25th, 2027
January 25th, 2024
January 25th, 2028
3.5072366313834147
January 25th, 2028
January 25th, 2024
January 25th, 2029
3.6057911572521055
January 25th, 2029
January 25th, 2024
January 25th, 2030
3.696356761258901
January 25th, 2030
January 25th, 2024
January 27th, 2031
3.7519625386279642
January 25th, 2031
January 25th, 2024
January 26th, 2032
3.798013860566534
January 25th, 2032
January 25th, 2024
January 25th, 2033
3.8648821096239376
January 25th, 2033
January 25th, 2024
January 25th, 2034
3.9095404321894733
January 25th, 2034
January 25th, 2024
January 25th, 2035
3.9774149897416455
January 25th, 2035
January 25th, 2024
January 25th, 2036
4.02808092779825
January 25th, 2036
January 25th, 2024
January 26th, 2037
4.0489491686

NameError: name 'figure' is not defined

In [104]:
a = 'GDP'
b = 'US'
c = '2024'
contrib1 = 'GS'
off = 'FED'

contrib = ['BAR', 'BOA', 'BNP', 'CE', 'CIT', 'CAG', 'CSU', 'DNS', 'FTC', 'GS', 'HSB', 'IG', 'JPM', 'MS', 'NTX', 'NS', 'NDA', 'PMA', 'UBS', 'WF', 'SCB' ]
today = ql.Date(datetime.datetime.now().day,datetime.datetime.now().month,datetime.datetime.now().year)
t_stg = bbg_date_str(today)
cal = ql.TARGET()
start = bbg_date_str(cal.advance(today, ql.Period( -1, ql.Years)))

if a == 'GDP':
    e = 'GD'
elif a == 'CPI':
    e = 'PI'
elif a == 'PCE':
    e = 'PC'
elif a == 'Core-PCE':
    e = 'CC'
elif a == 'UNEMP':
    e = 'UP'
elif a == 'FISC':
    e = 'BB'

g1 = ["EC"+e+b+" "+c[-2:]+" "+i+" Index" for i in contrib]

df1 = con.bdh(g1, 'PX_LAST',start, t_stg, longdata=True)
df1['date'] = [df1['date'][i].date() for i in np.arange(len(df1))]
df2 = con.bdh("EC"+e+b+" "+c[-2:]+" "+off+" Index", 'PX_LAST',start, t_stg, longdata=True)

d1 = [ql_to_datetime(cal.advance(today, ql.Period( int(-10-i), ql.Days))) for i in np.arange(240)]
m1 = [df1[(df1['date'] > d1[i]) & (df1['date'] < ql_to_datetime(cal.advance(today, ql.Period( int(0-i), ql.Days)))) ]['value'].mean()  for i in np.arange(len(d1))]

c1 = df1['ticker'].unique()

#HoverTool(tooltips=[('date', '@DateTime{%F}')],formatters={'@DateTime': 'datetime'})

s1 = figure(width=1000, height=400, tools = ["pan","wheel_zoom","box_zoom","save","reset","help"], toolbar_location='left', x_axis_type='datetime')
s1.xaxis.formatter=DatetimeTickFormatter(days="%d-%b-%y", months="%d-%b-%y")
s1.add_tools(HoverTool(tooltips=[('date', '$x{%b-%y}'), ('y', '$y')],
                       formatters={'$x': 'datetime'}))
s1.xgrid.visible = False
s1.ygrid.visible = False
s1.add_layout(Legend(), 'right')

for i in np.arange(len(c1)):
    if c1[i].split()[2] == contrib1:
        c2 = 'red'
        c3 = contrib1+": "+str(df1[df1['ticker'] == "EC"+e+b+" "+c[-2:]+" "+contrib1+" Index"].iloc[-1]['value'])
    else:
        c2 = 'silver'
        c3 = ''
    a1 = df1[df1['ticker'] == c1[i]]
    s1.line(a1['date'], a1['value'], legend_label=c3, color=c2, alpha=1.0, muted_alpha = 0.25)

    s1.legend.label_text_font = "calibri"
    s1.legend.label_text_font_size = "9pt"
    s1.legend.spacing = 1
    s1.legend.click_policy = "mute"
    s1.xaxis.axis_label = 'Value'
    s1.yaxis.axis_label = 'Date'

s1.line(df2['date'], df2['value'], color = 'forestgreen', legend_label = off+": "+str(df2['value'][len(df2)-1]), alpha=1.0)
s1.line( np.array(d1), np.array(m1), color = 'blue', legend_label = 'Avg: '+str(np.round(m1[0],2)), alpha=1.0)
s1.title.text = b+" "+a+" forecast: "+c
s1.title.text_font = "calibri"
s1.title.text_font_size = '10pt'
s1.title.align = 'left'

layout = row(s1)
show(layout)







In [28]:
rpi1 = infl_zc_swap_build('UKRPI', b=0)
rpi2 = infl_zc_swap_build('UKRPI', b=-1)


In [33]:

c1 = ['UKRPI']
h1 = [0,-10]
bar_chg = 1
sprd = 0
max_tenor = 30
built_curve = [rpi1, rpi2]
name = ''

n_ccy = len(c1)
n_chg = len(h1)

os.chdir('C:\\Users\A00007579\PycharmProjects\pythonProject')
today = ql.Date(datetime.datetime.now().day,datetime.datetime.now().month,datetime.datetime.now().year)
if built_curve == 0:
    crv_list = {}
    for k in np.arange(n_ccy):
        c2 = ccy_infl(c1[k], today)
        crv_list[c1[k]] = [infl_zc_swap_build( c1[k] ,i ,base_month_offset=0) for i in h1]

    crv = flat_lst(list(crv_list.values()))
else:
    crv = built_curve

ref_dates = [crv[i].ref_date for i in np.arange(len(crv))]

    ###### define number of subplots and number of objects !!!!!
if ((bar_chg == 0 ) & (sprd == 0)):
    n_plots = 1
    n_obj = {'curve': [n_ccy*n_chg]}
elif ((bar_chg == 1 ) & (sprd == 0)):
    if (n_chg < 3):
        n_plots = 2
        n_obj = {'curve':[n_ccy*n_chg], 'chg':[n_ccy]}

    elif ((n_ccy == 1) & (n_chg > 3)):
        n_plots = 2
        n_obj = {'curve': [n_ccy*n_chg], 'chg': [n_chg-1]}
    else:
        n_plots = 1 + n_ccy
        n_obj = {'curve': [n_ccy*n_chg], 'chg' : [n_chg-1]*n_ccy }

elif ((bar_chg == 0 ) & (sprd == 1)):
    n_plots = 2
    n_obj = {'curve ': [n_ccy*n_chg], 'chg': [n_ccy-1] }

else:
    if (n_chg < 3):
        n_plots = 3
        n_obj = {'curve': [n_ccy*n_chg], 'sprd': [n_ccy-1], 'chg': [n_ccy-1] }
    elif ((n_ccy == 2) & (n_chg > 3)):
        n_plots = 3
        n_obj = { 'curve': [n_ccy*n_chg], 'sprd': [n_ccy-1], 'chg': [n_chg-1] }
    else:
        n_plots = 1 + n_ccy
        n_obj = { 'curve': [n_ccy*n_chg], 'sprd' : [n_ccy-1],  'chg' : [n_chg-1]*(n_ccy-1) }

### Get ALL Data
inf_bases_dict = dict([(key, []) for key in c1])
for i in np.arange(len(crv)):
    infl_base = crv[i].base_month
    inf_bases_dict[c1[int(np.floor(i/len(h1)))]].append(infl_base)

rates = dict([(key, []) for key in c1])
for i in np.arange(len(crv)):
    yr_axis = np.arange(max_tenor)
    rates_c = []
    j = 0
    while j < max_tenor:
        start_sw = inf_bases_dict[list(inf_bases_dict.keys())[int(np.floor(i/len(h1)))]][0] +  ql.Period(str(j)+"Y")
        rates_c.append(Infl_ZC_Pricer(crv[i], start_sw, 1, lag = 0).zc_rate)
        j+=1
    rates[c1[int(np.floor(i/len(h1)))]].append(rates_c)

rates_change = dict([(key, []) for key in c1])
for j in rates.keys():
    k = 1
    while k < n_chg:
        rates_diff = 100*( np.array(rates[j][0]) - np.array(rates[j][k]))
        rates_change[j].append(rates_diff.tolist())
        k+=1
bar_dict = rates_change

if sprd == 1:
    c2 = [c1[0]+' - '+c1[i] for i in np.arange(1,n_ccy)]
    spreads = dict([(key, []) for key in c2])
    for i,j in enumerate(spreads.keys()):
        spreads[j] = 100*(np.array(rates[list(rates.keys())[0]]) - np.array(rates[list(rates.keys())[i+1]]))

    spreads_change = dict([(key, []) for key in c2])
    for j in spreads.keys():
        k = 1
        while k < n_chg:
            sprd_chg = 1*( np.array(spreads[j][0]) - np.array(spreads[j][k]))
            spreads_change[j].append(sprd_chg.tolist())
            k+=1
    bar_dict = spreads_change

### Plot Curve
s_plot = []
s1 = figure(width=1000, height=400, tools = ["pan","hover","wheel_zoom","box_zoom","save","reset","help"], toolbar_location='left')
s1.xgrid.visible = False
s1.ygrid.visible = False
s1.add_layout(Legend(), 'right')

for i in rates.keys():
    [s1.line(yr_axis+1, rates[i][j],
             legend_label = i+': '+str(ref_dates[j]),
             line_width = 2,
             color =  Category20[20][ 2*(list(rates.keys()).index(i)+ (j* (len(list(rates.keys())) == 1))) ], alpha = (1.0-(0.4*j)),
             muted_color = Category20[20][ 2*(list(rates.keys()).index(i)+ (j* (len(list(rates.keys())) == 1))) ], muted_alpha=0.2) for j in np.arange(n_chg)]

s1.legend.label_text_font = "calibri"
s1.legend.label_text_font_size = "9pt"
s1.legend.spacing = 1
s1.legend.click_policy = "mute"
s1.xaxis.axis_label = 'ZC_Rate'
s1.yaxis.axis_label = 'Tenor'
s_plot.append(s1)

### Plot Sprd
if sprd == 1:
    s2 = figure(width=1000, height=300, tools = ["pan","hover","wheel_zoom","box_zoom","save","reset","help"], toolbar_location='left')
    s2.xgrid.grid_line_dash = 'dotted'
    s2.ygrid.grid_line_dash = 'dotted'
    s2.add_layout(Legend(), 'right')
    for j in spreads.keys():
        s2.line( yr_axis+1 , spreads[j][0],
                     line_width = 2,
                     legend_label = str(j),
                     color = Category20[20][2*list(bar_dict.keys()).index(j)], alpha = 1.0,
                     muted_color = Category20[20][2*list(bar_dict.keys()).index(j)], muted_alpha=0.2)
    s2.legend.label_text_font = "calibri"
    s2.legend.label_text_font_size = "9pt"
    s2.legend.spacing = 1
    s2.legend.click_policy = "mute"
    s2.xaxis.axis_label = 'Spread'
    s2.yaxis.axis_label = 'Tenor'
    s_plot.append(s2)

### Plot Chg
if sprd == 1:
    start_sub_chg = 2
else:
    start_sub_chg = 1

if bar_chg == 1:
    n_sub_chg = len(n_obj['chg'])
    if n_sub_chg == 1:
        s3 = figure(width=1000, height=300, tools = ["pan","hover","wheel_zoom","box_zoom","save","reset","help"], toolbar_location='left')
        s3.xgrid.grid_line_dash = 'dotted'
        s3.ygrid.grid_line_dash = 'dotted'
        s3.add_layout(Legend(), 'right')
        width = 0.15
        bar_yr_axis = yr_axis
        for i in bar_dict.keys():
            for j in np.arange(len(bar_dict[i])):
                s3.vbar(x=np.array(bar_yr_axis)+1, top=bar_dict[i][j],
                            legend_label = i+': '+str(h1[j+1]),
                            width = width,
                            color = Category20[20][ 2*(list(bar_dict.keys()).index(i)+ (j* (len(list(bar_dict.keys())) == 1))) ], alpha = (1.0-(0.4*j)),
                            muted_color = Category20[20][ 2*(list(bar_dict.keys()).index(i)+ (j* (len(list(bar_dict.keys())) == 1))) ], muted_alpha=0.2)
                bar_yr_axis = bar_yr_axis+width+0.1
        s3.legend.label_text_font = "calibri"
        s3.legend.label_text_font_size = "9pt"
        s3.legend.spacing = 1
        s3.legend.click_policy = "mute"
        s3.xaxis.axis_label = 'Chg'
        s3.yaxis.axis_label = 'Tenor'
        s_plot.append(s3)

    else:
        for j in np.arange(n_sub_chg):
            s4 = figure(width=1000, height=300, tools = ["pan","hover","wheel_zoom","box_zoom","save","reset","help"], toolbar_location='left')
            s4.xgrid.grid_line_dash = 'dotted'
            s4.ygrid.grid_line_dash = 'dotted'
            s4.add_layout(Legend(), 'right')
            width = 0.15
            bar_yr_axis = yr_axis
            for i in np.arange(len(bar_dict[ list(bar_dict.keys())[j]  ])):
                s4.vbar(x=np.array(bar_yr_axis)+1, top=bar_dict[list(bar_dict.keys())[j]][i],
                            legend_label= list(bar_dict.keys())[j]+': '+str(h1[i+1]),
                            width = width,
                            color = Category20[20][(j+(2*i)+1)],alpha = 1.0,
                            muted_color = Category20[20][(j+(2*i)+1)], muted_alpha=0.2)
                bar_yr_axis = bar_yr_axis+width+0.1
                s4.legend.label_text_font = "calibri"
                s4.legend.label_text_font_size = "9pt"
                s4.legend.spacing = 1
                s4.legend.click_policy = "mute"
                s_plot.append(s4)



layout = column(*s_plot)
show(layout)




















In [106]:
p1 = ['S0490FS 1Y1Y BLC Curncy']
p2 = ['USOSFR10 Curncy']
p3 = ['S0490FS 2Y1Y BLC Curncy', 'S0490FS 3Y1Y BLC Curncy']
p4 = ['USOSFR5 Curncy', 'USOSFR30 Curncy']


In [144]:
st = sofr.ref_date
et = sofr.cal.advance(st, ql.Period('1Y'))
schedule = ql.MakeSchedule(st, et, ql.Period('1W'))

[ql_to_datetime(schedule[int(i)]) for i in np.arange(len(schedule))]

fwd_curve = [Swap_Pricer([[sofr, ql_to_datetime(schedule[int(i)]).strftime('%d-%m-%Y'),5],[sofr,ql_to_datetime(schedule[int(i)]).strftime('%d-%m-%Y'),30]]).spread for i in np.arange(len(schedule))]

In [244]:

def plot_tool_bbg(a, crv):
#    a = [2,5,10]
#    crv = ois_dc_build('SOFR_DC', b=0)

    min_y = []
    max_y = []

    v_reg = []
    fwd_reg = []
    lab_reg = []

    s1 = figure(width=550, height=275, tools = ["pan",'crosshair',"wheel_zoom","box_zoom","save","reset","help"], toolbar_location='right')
    s1.xgrid.visible = False
    s1.ygrid.visible = False
    s1.xaxis.formatter=DatetimeTickFormatter(days="%d-%b-%y", months="%d-%b-%y")
#    s1.add_tools(HoverTool(tooltips=[('date', '$x{%d.%b.%y}'), ('y', '$y')], formatters={'$x': 'datetime'}))

    for j in np.arange(len(a)):
        print(j)

        if len(a[j]) == 4:
            tk = []
            if a[j][0] == 0:
                tk.append(crv.bbgplot_tickers[0]+str(a[j][1])+' Curncy')
            else:
                tk.append(crv.bbgplot_tickers[1]+' '+str(a[j][0])+'Y'+str(a[j][1])+'Y'+ ' BLC Curncy')
            if a[j][3] != 0:
                if a[j][2] ==0:
                    tk.append(crv.bbgplot_tickers[0]+str(a[j][3]) + ' Curncy')
                else:
                    tk.append(crv.bbgplot_tickers[1] + ' ' + str(a[j][2]) + 'Y' + str(a[j][3]) + 'Y' + ' BLC Curncy')
            print(tk)

            x1 = con.bdh(tk , 'PX_LAST', '20210101', datetime.now().strftime('%Y%m%d'))
            x1 = x1.dropna()

            if len(tk) == 1:
                y = x1[(tk[0], 'PX_LAST')]
                lab1 = tk[0].split(' ')[len(tk[0].split(' '))//3]
            else:
                y = 100*(x1.iloc[:,1] - x1.iloc[:,0])
                lab1 = ' - '.join([tk[i].split(' ')[len(tk[i].split(' '))//3] for i in np.arange(len(tk))])

            v_reg.append(y)
            lab_reg.append(lab1)
            ### mkt pricing
            t1 = crv.ref_date
            t2 = crv.cal.advance(t1, ql.Period('1Y'))
            schedule = ql.MakeSchedule(t1, t2, ql.Period('1W'))
            sch2 = [ datetime.combine(ql_to_datetime(schedule[int(i)]), time()) for i in np.arange(len(schedule))]

            if len(tk) == 1:
                fwd_curve = [Swap_Pricer([[crv, ql_to_datetime( crv.cal.advance(schedule[int(i)], ql.Period(str(a[j][0])+'Y'))  ).strftime('%d-%m-%Y')  ,a[j][1]]]).rate[0] for i in np.arange(len(schedule))]
            else:
                fwd_curve = [Swap_Pricer([[crv, ql_to_datetime(  crv.cal.advance(schedule[int(i)], ql.Period(str(a[j][0])+'Y'))   ).strftime('%d-%m-%Y'),a[j][1]],
                                          [crv, ql_to_datetime(  crv.cal.advance(schedule[int(i)], ql.Period(str(a[j][2])+'Y'))   ).strftime('%d-%m-%Y'),a[j][3]]]).spread for i in np.arange(len(schedule))]
            fwd_reg.append(fwd_curve)

            min_y.append(min(min(fwd_curve),min(y)))
            max_y.append(max(max(fwd_curve),max(y)))
            ratio = (max_y[0] - min_y[0]) / (max_y[j] - min_y[j])
            print('min_y: ', min_y, 'max_y: ', max_y, 'ratio: ', ratio)
            z = [ min_y[0]  + (y[k] - min_y[j])*ratio for k in np.arange(len(y))]
            fwd_curve_z = [ min_y[0]  + (fwd_curve[k] - min_y[j])*ratio for k in np.arange(len(fwd_curve))]

            s1.add_tools(HoverTool(tooltips=[('date', '$x{%d.%b.%y}'), ('y', '@ht')], formatters={'$x': 'datetime'}))

            s1.line('x', 'y', legend_label=lab1 , color=Category20[20][2*j], alpha=1.0, muted_alpha = 0.25, source = ColumnDataSource( data=dict(x= list(x1.index), y=z, ht=y)))
            s1.line('x', 'y', legend_label=lab1 , color=Category20[20][(2*j)+1], alpha=0.8, muted_alpha = 0.25, source = ColumnDataSource( data=dict(x= sch2, y=fwd_curve_z, ht=fwd_curve)))

        else:
            tk=[]
            for i in np.arange(len(a[j])):
                if a[j][i] != 0:
                    tk.append(crv.bbgplot_tickers[0]+str(a[j][i])+' Curncy')
            print(tk)

            x1 = con.bdh(tk, 'PX_LAST', '20210101', datetime.now().strftime('%Y%m%d'))
#            print(x1)
            x1 = x1.dropna()

            if len(tk) == 2:
                y = 100 * (x1.iloc[:, 1] - x1.iloc[:, 0])
            else:
                y = -100*(x1.iloc[:,2] + x1.iloc[:,0] - 2* x1.iloc[:,1])
            lab1 = '-'.join([str(a[j][i]) for i in np.arange(len(tk))])

            v_reg.append(y)
            lab_reg.append(lab1)
            ### mkt pricing
            t1 = crv.ref_date
            t2 = crv.cal.advance(t1, ql.Period('1Y'))
            schedule = ql.MakeSchedule(t1, t2, ql.Period('1W'))
            sch2 = [datetime.combine(ql_to_datetime(schedule[int(i)]), time()) for i in
                    np.arange(len(schedule))]

            if len(tk) == 2:
                fwd_curve = [Swap_Pricer([[crv, ql_to_datetime( schedule[int(i)]).strftime('%d-%m-%Y'), a[j][0]],
                                          [crv, ql_to_datetime( schedule[int(i)]).strftime('%d-%m-%Y'), a[j][1]]]).spread for i in np.arange(len(schedule))]
            else:
                fwd_curve = [Swap_Pricer([[crv, ql_to_datetime(schedule[int(i)]).strftime('%d-%m-%Y'), a[j][0]],
                                          [crv, ql_to_datetime(schedule[int(i)]).strftime('%d-%m-%Y'), a[j][1]],
                                          [crv, ql_to_datetime(schedule[int(i)]).strftime('%d-%m-%Y'), a[j][2]]]).fly for i in np.arange(len(schedule))]
            fwd_reg.append(fwd_curve)

            min_y.append(min(min(fwd_curve),min(y)))
            max_y.append(max(max(fwd_curve),max(y)))
            ratio = (max_y[0] - min_y[0]) / (max_y[j] - min_y[j])
            print('min_y: ', min_y, 'max_y: ', max_y, 'ratio: ', ratio)
            z = [ min_y[0]  + (y[k] - min_y[j])*ratio for k in np.arange(len(y))]
            fwd_curve_z = [ min_y[0]  + (fwd_curve[k] - min_y[j])*ratio for k in np.arange(len(fwd_curve))]

            s1.add_tools(HoverTool(tooltips=[('date', '$x{%d.%b.%y}'), ('y', '@ht')], formatters={'$x': 'datetime'}))

            s1.line('x', 'y', legend_label=lab1, color=Category20[20][2*j], alpha=1.0, muted_alpha=0.25, source = ColumnDataSource( data=dict(x= list(x1.index), y=z, ht=y)  ))
            s1.line('x', 'y', legend_label=lab1, color=Category20[20][(2*j)+1], alpha=0.8, muted_alpha=0.25, source = ColumnDataSource( data=dict(x= sch2, y=fwd_curve_z, ht=fwd_curve)))


        s1.legend.label_text_font = "calibri"
        s1.legend.label_text_font_size = "9pt"
        s1.legend.location = 'top_left'
        s1.legend.click_policy = "mute"
        s1.legend.spacing = 1
        s1.yaxis.major_label_text_color = Category20[20][0]
        s1.yaxis.axis_line_color = "red"

        s1.xaxis.axis_label = 'Date'
        s1.yaxis.axis_label = 'Rate'

    if len(a) == 2:
#        print('v_reg:', v_reg)
#        print('len_v_reg ===', len(v_reg))
        df_x1 = pd.DataFrame(v_reg[0])
        df_x2 = pd.DataFrame(v_reg[1])
        df_reg = pd.merge(df_x1, df_x2, left_index=True, right_index=True)
        df_reg = df_reg.dropna()

        x = df_reg.iloc[:,0]
        y = df_reg.iloc[:,1]
#        print('x_reg', x)
#        print('y_reg', y)

        tab1 = TabPanel(child=s1, title="plot")

        slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
        y_reg_fit = intercept + (slope * x)
        residuals = y - y_reg_fit
#        print('x_fwd:', fwd_reg[0])
#        print('y_fwd:', fwd_reg[1])
#        print('int:', intercept, 'slope:', slope)
        residuals_fwds = fwd_reg[1] - intercept - (slope * np.array(fwd_reg[0]))

        s2 = figure(width=800, height=425, tools = ["pan",'crosshair',"wheel_zoom","box_zoom","save","reset","help"], toolbar_location='right')
        s2.add_tools(HoverTool(tooltips=[('x', '$x'), ('y', '$y')]))
        s2.xgrid.visible = False
        s2.ygrid.visible = False
        s2_alphas = np.linspace(0.1, 1.0, len(df_reg))
        s2_source = ColumnDataSource(data=dict(x=x, y=y, s2_alphas=s2_alphas))
        s2.circle('x', 'y', size=5, alpha='s2_alphas', color="navy", source=s2_source)
        s2.circle(fwd_reg[0], fwd_reg[1], size=5, alpha=0.9, color="darkseagreen")
        s2.line(x, y_reg_fit, color="red")
        s2.xaxis.axis_label = lab_reg[0]
        s2.yaxis.axis_label = lab_reg[1]
        tab2 = TabPanel(child=s2, title="regression")


        s3 = figure(width=800, height=425, tools = ["pan",'crosshair',"wheel_zoom","box_zoom","save","reset","help"], toolbar_location='right')
        s3.xgrid.visible = False
        s3.ygrid.visible = False
        s3.xaxis.formatter=DatetimeTickFormatter(days="%d-%b-%y", months="%d-%b-%y")
        s3.circle(df_reg.index, residuals, size=2)
        s3.circle(sch2, residuals_fwds, size=2, color = 'darkseagreen', alpha=0.3)
        s3.line([min(df_reg.index), max(sch2)], [0, 0], color="lightsteelblue", line_width=1)  # Zero line for reference
        tab3 = TabPanel(child=s3, title="residuals")

        tabs = Tabs(tabs=[tab1, tab2, tab3])
        show(tabs)
    else:
        show(s1)

    return


In [245]:
plot_tool_bbg([ [1,1,0,0], [2,5,10] ], sofr)



0
['S0490FS 1Y1Y BLC Curncy']
min_y:  [0.058] max_y:  [4.72892] ratio:  1.0
1
['USOSFR2 Curncy', 'USOSFR5 Curncy', 'USOSFR10 Curncy']
min_y:  [0.058, -71.71000000000004] max_y:  [4.72892, 30.95000000000003] ratio:  0.045498928501850736


In [166]:
#d6 = pd.DataFrame(d5[1])
#d7 = pd.DataFrame(d5[3])

#d8 = pd.merge(d6, d7, left_index=True, right_index=True)
#d8 = d8.dropna()
d5.iloc[:,0]


date
2021-01-04   15.763999999999998
2021-01-05   19.465000000000003
2021-01-06               24.136
2021-01-07   25.568999999999996
2021-01-08                27.91
                    ...        
2024-01-31    5.064000000000002
2024-02-01    3.232999999999997
2024-02-02   1.6259999999999941
2024-02-05    1.684999999999981
2024-02-06   0.5139999999999922
Name: 0_x, Length: 783, dtype: float64

In [75]:
x1 = con.bdh(['S0490FS 2Y1Y BLC Curncy', 'S0490FS 3Y1Y BLC Curncy'], 'PX_LAST', '20210101', datetime.now().strftime('%Y%m%d'))
x1 = x1.dropna()
x2 = con.bdh(['USOSFR2 Curncy', 'USOSFR5 Curncy', 'USOSFR10 Curncy'], 'PX_LAST', '20210101', datetime.now().strftime('%Y%m%d'))
x2 = x2.dropna()

x3 = [x1.index, x1.iloc[:,1], x2.index, x2.iloc[:,0], x2.iloc[:,2]]

x4 = pd.DataFrame(x3).transpose()

x5 = x4.dropna()
x5

,0,1,2,3,4
0,2021-01-01,0.33234,2021-01-04,0.058,0.6937
1,2021-01-04,0.30243,2021-01-05,0.058,0.736
2,2021-01-05,0.36119,2021-01-06,0.0715,0.811
3,2021-01-06,0.43177,2021-01-07,0.0831,0.8647
4,2021-01-07,0.50071,2021-01-08,0.0804,0.9096
...,...,...,...,...,...
778,2023-12-27,3.06017,2024-01-31,4.0715,3.5487
779,2023-12-28,3.1062,2024-02-01,4.0579,3.5208
780,2023-12-29,3.12715,2024-02-02,4.237,3.6605
781,2024-01-01,3.12715,2024-02-05,4.3362,3.7905


In [81]:
##### old tool  #########

def plot_tool_bbg(a, crv):
#    a = [2,5,10]
#    crv = ois_dc_build('SOFR_DC', b=0)

    if len(a) == 4:
        tk = []
        if a[0] == 0:
            tk.append(crv.bbgplot_tickers[0]+str(a[1])+' Curncy')
        else:
            tk.append(crv.bbgplot_tickers[1]+' '+str(a[0])+'Y'+str(a[1])+'Y'+ ' BLC Curncy')
        if a[3] != 0:
            if a[2] ==0:
                tk.append(crv.bbgplot_tickers[0]+str(a[3]) + ' Curncy')
            else:
                tk.append(crv.bbgplot_tickers[1] + ' ' + str(a[2]) + 'Y' + str(a[3]) + 'Y' + ' BLC Curncy')
        print(tk)

        x1 = con.bdh(tk , 'PX_LAST', '20210101', '20240131')

        s1 = figure(width=550, height=275, tools = ["pan",'crosshair',"wheel_zoom","box_zoom","save","reset","help"], toolbar_location='right')
        s1.xgrid.visible = False
        s1.ygrid.visible = False
        s1.xaxis.formatter=DatetimeTickFormatter(days="%d-%b-%y", months="%d-%b-%y")
        s1.add_tools(HoverTool(tooltips=[('date', '$x{%d.%b.%y}'), ('y', '$y')], formatters={'$x': 'datetime'}))

        if len(tk) == 1:
            y = x1[(tk[0], 'PX_LAST')]
            lab1 = tk[0].split(' ')[len(tk[0].split(' '))//3]
        else:
            y = 100*(x1.iloc[:,1] - x1.iloc[:,0])
            lab1 = ' - '.join([tk[i].split(' ')[len(tk[i].split(' '))//3] for i in np.arange(len(tk))])

        ### mkt pricing
        t1 = crv.ref_date
        t2 = crv.cal.advance(t1, ql.Period('1Y'))
        schedule = ql.MakeSchedule(t1, t2, ql.Period('1W'))
        sch2 = [ datetime.datetime.combine(ql_to_datetime(schedule[int(i)]), datetime.time()) for i in np.arange(len(schedule))]

        if len(tk) == 1:
            fwd_curve = [Swap_Pricer([[crv, ql_to_datetime( crv.cal.advance(schedule[int(i)], ql.Period(str(a[0])+'Y'))  ).strftime('%d-%m-%Y')  ,a[1]]]).rate for i in np.arange(len(schedule))]
        else:
            fwd_curve = [Swap_Pricer([[crv, ql_to_datetime(  crv.cal.advance(schedule[int(i)], ql.Period(str(a[0])+'Y'))   ).strftime('%d-%m-%Y'),a[1]],
                                      [crv, ql_to_datetime(  crv.cal.advance(schedule[int(i)], ql.Period(str(a[2])+'Y'))   ).strftime('%d-%m-%Y'),a[3]]]).spread for i in np.arange(len(schedule))]


        s1.line(x1.index, y, legend_label=lab1 , color='red', alpha=1.0, muted_alpha = 0.25)
        s1.line(sch2, fwd_curve, legend_label=lab1 , color='blue', alpha=0.8, muted_alpha = 0.25)

    else:
        tk=[]
        for i in np.arange(len(a)):
            if a[i] != 0:
                tk.append(crv.bbgplot_tickers[0]+str(a[i])+' Curncy')
        print(tk)

        x1 = con.bdh(tk, 'PX_LAST', '20210101', '20240131')

        s1 = figure(width=550, height=275,
                    tools=["pan", 'crosshair', "wheel_zoom", "box_zoom", "save", "reset", "help"],
                    toolbar_location='right')
        s1.xgrid.visible = False
        s1.ygrid.visible = False
        s1.xaxis.formatter = DatetimeTickFormatter(days="%d-%b-%y", months="%d-%b-%y")
        s1.add_tools(HoverTool(tooltips=[('date', '$x{%d.%b.%y}'), ('y', '$y')], formatters={'$x': 'datetime'}))

        if len(tk) == 2:
            y = 100 * (x1.iloc[:, 1] - x1.iloc[:, 0])
        else:
            y = -100*(x1.iloc[:,2] + x1.iloc[:,0] - 2* x1.iloc[:,1])
        lab1 = '-'.join([str(a[i]) for i in np.arange(len(tk))])

        s1.line(x1.index, y, legend_label=lab1, color='green', alpha=1.0, muted_alpha=0.25)

        ### mkt pricing
        t1 = crv.ref_date
        t2 = crv.cal.advance(t1, ql.Period('1Y'))
        schedule = ql.MakeSchedule(t1, t2, ql.Period('1W'))
        sch2 = [datetime.datetime.combine(ql_to_datetime(schedule[int(i)]), datetime.time()) for i in
                np.arange(len(schedule))]

        if len(tk) == 2:
            fwd_curve = [Swap_Pricer([[crv, ql_to_datetime( schedule[int(i)]).strftime('%d-%m-%Y'), a[0]],
                                      [crv, ql_to_datetime( schedule[int(i)]).strftime('%d-%m-%Y'), a[1]]]).spread for i in np.arange(len(schedule))]
        else:
            fwd_curve = [Swap_Pricer([[crv, ql_to_datetime(schedule[int(i)]).strftime('%d-%m-%Y'), a[0]],
                                      [crv, ql_to_datetime(schedule[int(i)]).strftime('%d-%m-%Y'), a[1]],
                                      [crv, ql_to_datetime(schedule[int(i)]).strftime('%d-%m-%Y'), a[2]]]).fly for i in np.arange(len(schedule))]

        s1.line(sch2, fwd_curve, legend_label=lab1, color='blue', alpha=0.8, muted_alpha=0.25)


    s1.legend.label_text_font = "calibri"
    s1.legend.label_text_font_size = "9pt"
    s1.legend.location = 'top_left'
    s1.legend.click_policy = "mute"
#    s1.xaxis.axis_label = 'Date'
#    s1.yaxis.axis_label = 'Rate'

    return [s1]



792
779


In [153]:
today = ql.Date(datetime.datetime.now().day, datetime.datetime.now().month, datetime.datetime.now().year)

str(datetime.datetime.now())[:10]

datetime.datetime.now().strftime('%Y%m%d')


'20240206'

In [13]:
now = datetime.now()
dates = [now - timedelta(days=x) for x in range(365)]
data = np.random.randn(365).cumsum()

#df = pd.DataFrame({'date': dates, 'value': data})
df =  con.bdh(p2 , 'PX_LAST', '20210101', '20240131', longdata=True)

# Function to create a time series plot
def create_time_series_plot(time_range):
    filtered_df = df[df['date'] > (now - timedelta(days=time_range))]
    p = figure(x_axis_type='datetime', title='Time Series Plot')
    p.line(filtered_df['date'], filtered_df['value'])
    return p

# Callback for updating the plot
def update_plot(event):
    time_range_dict = {'1m': 30, '3m': 90, '6m': 180, '1y': 365}
    new_plot = create_time_series_plot(time_range_dict[event.new])
    time_series_pane.object = new_plot

# Radio button group for selecting time range
time_range_buttons = pn.widgets.RadioButtonGroup(name='Time Range', options=['1m', '3m', '6m', '1y'], button_type='success')
time_range_buttons.param.watch(update_plot, 'value')

# Initial plot
initial_plot = create_time_series_plot(365)
time_series_pane = pn.pane.Bokeh(initial_plot)

# Layout
layout = pn.Column(time_range_buttons, time_series_pane)
layout.servable()

NameError: name 'pn' is not defined

In [181]:
df = pd.DataFrame([[3,2,1],[3,2,1],[3,2,1]])

In [182]:
df

,0,1,2
0,3,2,1
1,3,2,1
2,3,2,1


In [190]:
df.iloc[0].tolist()

[3, 2, 1]

In [255]:

datetime.combine( datetime.now().date(), datetime.min.time()).strftime('%Y%m%d')

'20240206'

In [4]:
today = ql.Date(datetime.datetime.now().day, datetime.datetime.now().month, datetime.datetime.now().year)
c = ccy('SOFR_DC', today)

ticker = 'USSOSR'
contrib = 'GFUS'
base_ticker = 'FDTR Index'
ois_fix = con.ref(c.fixing, 'PX_LAST')['value'][0]
base_fix = con.ref(base_ticker, 'PX_LAST')['value'][0]
basis = base_fix - ois_fix

### Get bbg tickers
t_list = [ticker+str(i)+' '+contrib+' Curncy' for i in range(1,16) ]
df1 = con.bdh(t_list, 'PX_LAST', bbg_date_str(today), bbg_date_str(today), longdata=True)
df1.ticker = df1.ticker.astype("category")
df1.ticker = df1.ticker.cat.set_categories(t_list)
df1 = df1.sort_values(["ticker"])
df1.reset_index(inplace=True, drop=True)

### Get FOMC Dates and set index
x = con.bulkref(base_ticker,'ECO_FUTURE_RELEASE_DATE_LIST')['value']
y = pd.Series([ pd.datetime(int(x[i][0:4]),int(x[i][5:7]),int(x[i][8:10])) for i in range(len(x)) ])
y = pd.DataFrame(y[y > (pd.datetime.now()+pd.DateOffset(days=-1))], columns = ['Meets'])
y1 = y['Meets'].append( pd.Series( [   y[-1:]['Meets'].values[0]+np.timedelta64(50*i,'D') for i in range(1, len(t_list)-len(y)+1 )] ) )
y2 = pd.DataFrame(y1)
meet_index = pd.Series( [y2.iloc[i,][0].strftime('%b') +'-'+ y2.iloc[i,][0].strftime('%y') for i in range(len((y2))) ] )

### Set df
df1['meet_date'] = meet_index
df1['cb'] = np.round(df1['value']+basis,3)
df1['step'] = 100*np.array([df1['cb'][0]-base_fix] +df1['cb'].diff()[1:].tolist())
df1['cum'] = df1['step'].cumsum()

df1


,date,ticker,field,value,meet_date,cb,step,cum
0,2024-02-07,USSOSR1 GFUS Curncy,PX_LAST,5.286,Mar-24,5.476,-2.400000000000002,-2.400000000000002
1,2024-02-07,USSOSR2 GFUS Curncy,PX_LAST,5.133,May-24,5.323,-15.299999999999958,-17.69999999999996
2,2024-02-07,USSOSR3 GFUS Curncy,PX_LAST,4.913,Jun-24,5.103,-22.000000000000064,-39.700000000000024
3,2024-02-07,USSOSR4 GFUS Curncy,PX_LAST,4.697,Jul-24,4.887,-21.60000000000002,-61.30000000000004
4,2024-02-07,USSOSR5 GFUS Curncy,PX_LAST,4.465,Sep-24,4.655,-23.199999999999932,-84.49999999999997
5,2024-02-07,USSOSR6 GFUS Curncy,PX_LAST,4.297,Nov-24,4.487,-16.800000000000015,-101.29999999999998
6,2024-02-07,USSOSR7 GFUS Curncy,PX_LAST,4.14,Dec-24,4.33,-15.700000000000003,-116.99999999999999
7,2024-02-07,USSOSR8 GFUS Curncy,PX_LAST,3.958,Jan-25,4.148,-18.20000000000004,-135.20000000000002
8,2024-02-07,USSOSR9 GFUS Curncy,PX_LAST,3.808,Mar-25,3.998,-14.999999999999947,-150.19999999999996
9,2024-02-07,USSOSR10 GFUS Curncy,PX_LAST,3.689,May-25,3.879,-11.900000000000022,-162.1


In [31]:
month = df1['meet_date']
step = np.round(df1['step'],1)
cum = np.round(df1['cum'],0)
fwd_base = df1['cb']
s1_source = ColumnDataSource(data=dict(x=month, y=step, z=cum, k=fwd_base))

s1 = figure(x_range=month, width=800, height=450, tools = ["pan", "wheel_zoom","box_zoom","save","reset","help"], toolbar_location='right')
s1.xgrid.visible = False
s1.ygrid.visible = False
s1.vbar(x='x', top='y', width=0.7, source=s1_source, color = 'lightsteelblue')
zero_line = Span(location=0, dimension='width', line_color='darkseagreen', line_width=1)
s1.renderers.extend([zero_line])
labels = LabelSet(x='x', y='y', text='y', level='glyph', text_align='center', y_offset=-16, source=s1_source, text_font_size='12px', text_color='midnightblue')
s1.add_layout((labels))

s2 = figure(x_range=month, width=800, height=450, tools = ["pan", "wheel_zoom","box_zoom","save","reset","help"], toolbar_location='right')
s2.xgrid.visible = False
s2.ygrid.visible = False
s2.vbar(x='x', top='z', width=0.7, source=s1_source, color = 'lightsteelblue')
zero_line = Span(location=0, dimension='width', line_color='darkseagreen', line_width=1)
s2.renderers.extend([zero_line])
labels_1 = LabelSet(x='x', y='z', text='z', level='glyph', text_align='center', y_offset=-16, source=s1_source, text_font_size='10px', text_color='midnightblue')
labels_2 = LabelSet(x='x', y='z', text='k', level='glyph', text_align='center', y_offset=10, source=s1_source, text_font_size='10px', text_color='firebrick')
s2.add_layout((labels_1))
s2.add_layout((labels_2))

tab1 = TabPanel(child=s1, title="step")
tab2 = TabPanel(child=s2, title="cum")
tabs = Tabs(tabs=[tab1, tab2])
show(tabs)



In [6]:
type(df1['meet_date'][0])

str

In [6]:
dt = get_wirp( [['SOFR_DC'], [datetime.date(2024, 2, 7), datetime.date(2023, 10, 18)]] )

[datetime.date(2024, 2, 7), datetime.date(2023, 10, 18)]


In [15]:

def plot_wirp(dt1, chg=0, sprd=0):
#    dt1 = dt
    n_crv = len(dt1.keys())
    crv = list(dt1.keys())
    n_dates = len(dt1[crv[0]])

    month = dict()
    step = dict()
    cum = dict()
    fwd_base = dict()
    month_chg = dict()
    step_chg = dict()
    cum_chg = dict()
#    print('n_crv:',n_crv)
#    print('crv:',crv)

    for j in crv:
#        print('j:', j)
#        print('crv[0]:', crv[0])
#        print('dt1[crv[0]]:', dt1[crv[0]])
        month[j] = [dt1[j][i]['meet_date'].tolist() for i in np.arange(n_dates)]
        step[j] =  [np.around(dt1[j][i]['step'],1).tolist() for i in np.arange(n_dates)]
        cum[j] =   [np.around(dt1[j][i]['cum'],0).tolist() for i in np.arange(n_dates)]
        fwd_base[j] = [dt1[j][i]['cb'].tolist() for i in np.arange(n_dates)]

    if n_dates >1:
        for j in crv:
            cut_off = month[j][1].index(month[j][0][0])
            month_chg[j] = [month[j][m][:cut_off]+month[j][0] for m in np.arange(1,n_dates)]
            step_chg[j] = [(np.round(100 * (5.5 - np.array(fwd_base[j][m][:cut_off])), 1)).tolist() +
                           (np.array(step[j][0][:-cut_off])-np.array(step[j][m][cut_off:])).tolist() +
                           cut_off*[0] for m in np.arange(1,n_dates)]

            cum_chg[j] = [list(accumulate(step_chg[j][m])) for m in np.arange(n_dates-1)]

    if n_crv != 1:
        itr = [max([len(step[j][0]) for j in crv]) - len(step[j][0]) for j in crv]
        for j in np.arange(len(itr)):
            for k in np.arange(n_dates):
                month[crv[j]][k] = month[crv[j]][k] + itr[j]*['n/a']
                step[crv[j]][k] = step[crv[j]][k] + itr[j]*[0]
                cum[crv[j]][k] = cum[crv[j]][k] + itr[j]*[cum[crv[j]][k][-1]]
                fwd_base[crv[j]][k] = fwd_base[crv[j]][k] + itr[j]*[fwd_base[crv[j]][k][-1]]

        itr2 = [max([len(step_chg[j][0]) for j in crv]) - len(step_chg[j][0]) for j in crv]
        for j in np.arange(len(itr2)):
            for k in np.arange(n_dates-1):
                month_chg[crv[j]][k] = month_chg[crv[j]][k] + itr2[j] * ['n/a']
                step_chg[crv[j]][k] = step_chg[crv[j]][k] + itr2[j] * [0]
                cum_chg[crv[j]][k] = cum_chg[crv[j]][k] + itr2[j] * [cum_chg[crv[j]][k][-1]]

    ### plot step only:
    if n_crv == 1:
        _month = month[crv[0]][0]
        _step = step[crv[0]][0]
        _cum = cum[crv[0]][0]
        _fwd_base = fwd_base[crv[0]][0]
        x_range_var = _month
        s1_source = ColumnDataSource(data=dict(x=_month, y=_step, z=_cum, k=_fwd_base))

        if chg == 1:
            print('in')
            _month_chg = month_chg[crv[0]][0]
            _step_chg = step_chg[crv[0]][0]
            _cum_chg = cum_chg[crv[0]][0]
            x_range_var = _month_chg
            itr3 = len(_step_chg)-len(_step)
            _month = ['1','2','3'] + _month
            _step = itr3*[0] + _step
            _cum = itr3*[0] + _cum
            _fwd_base = itr3*[5.5] + _fwd_base
            print('*******_ month',_month)
            print('*******_ step', _step)
            print('*******_ step_chg', _step_chg)
            s1_source = ColumnDataSource(data=dict(x=_month, y=_step, z=_cum, k=_fwd_base, x2=_month_chg, y2=_step_chg, z2=_cum_chg))

#        s_plot = []
        print('out')
        s1 = figure(x_range=x_range_var, width=525, height=400, tools=["pan", "wheel_zoom", "box_zoom", "save", "reset", "help"], toolbar_location='right')
        s1.xgrid.visible = False
        s1.ygrid.visible = False
        s1.vbar(x='x', top='y', width=0.7, source=s1_source, color='lightsteelblue')
        zero_line = Span(location=0, dimension='width', line_color='darkseagreen', line_width=1)
        s1.renderers.extend([zero_line])
        labels_1 = LabelSet(x='x', y='y', text='y', level='glyph', text_align='center', y_offset=-16, source=s1_source,
                            text_font_size='10px', text_color='midnightblue')
        s1.add_layout((labels_1))
        s1.xaxis.major_label_orientation = math.pi / 4
        s1.y_range = Range1d(min(_step) - 5, max(_step) + 5)
#        s1.circle(x='x2', y='y2', size=10, source=s1_source, color='firebrick', alpha=0.7, legend_label='chg')
#        show(s1)


        if chg == 1:
#            s2 = figure(x_range=_month_chg, width=525, height=400, tools=["pan", "wheel_zoom", "box_zoom", "save", "reset", "help"], toolbar_location='right')
#            print('_month_chg:',_month_chg)
#            print('_step_chg:', _step_chg)
#            s2_source = ColumnDataSource(data=dict(x=_month_chg, y=_step_chg, z=_cum_chg))
#            s2.xgrid.visible = False
#            s2.ygrid.visible = False
            s1.circle(x='x2', y='y2', size=10, source=s1_source, color = 'firebrick', alpha=0.7, legend_label = 'chg')
#            zero_line = Span(location=0, dimension='width', line_color='darkseagreen', line_width=1)
#            s2.renderers.extend([zero_line])
            labels_1a = LabelSet(x='x2', y='y2', text='y2', level='glyph', text_align='center', y_offset=-16, source=s1_source, text_font_size='8px', text_color='midnightblue')
            s1.add_layout((labels_1a))
#            s2.xaxis.major_label_orientation = math.pi / 4
#            s2.y_range = Range1d(min(_step) - 5, max(_step) + 5)
#            s_plot.append(s2)
            s1.legend.location = 'bottom_right'
            s1.legend.label_text_font = "calibri"
            s1.legend.label_text_font_size = "9pt"
            s1.legend.spacing = 1
            s1.legend.click_policy = "mute"
#        s_plot.append(s1)
#        show(s1)

        s3 = figure(x_range=_month, width=525, height=400, tools=["pan", "hover", "wheel_zoom", "box_zoom", "save", "reset", "help"], toolbar_location='right')
        s3.xgrid.visible = False
        s3.ygrid.visible = False
        s3.vbar(x='x', top='z', width=0.7, source=s1_source, color='lightsteelblue')
        zero_line = Span(location=0, dimension='width', line_color='darkseagreen', line_width=1)
        s3.renderers.extend([zero_line])
        labels_1 = LabelSet(x='x', y='z', text='z', level='glyph', text_align='center', y_offset=-16, source=s1_source,
                            text_font_size='10px', text_color='midnightblue')
        s3.add_layout((labels_1))
        s3.xaxis.major_label_orientation = math.pi / 4
        s3.y_range = Range1d(min(_cum) - 15, max(_cum) + 15)

        tab1 = TabPanel(child=s1, title="step")
        tab2 = TabPanel(child=s3, title="cum")
        tabs = Tabs(tabs=[tab1, tab2])
    else:
        print(n_crv, crv, month)
        n_grps = max([ len(month[crv[i]][0]) for i in np.arange(n_crv)])
        grps = np.arange(1,n_grps+1)
        step_data = pd.DataFrame()
        cum_data = pd.DataFrame()
        fwd_data = pd.DataFrame()
        for j in crv:
            step_data[j] =  step[j][0]
            cum_data[j] = cum[j][0]
            fwd_data[j] = fwd_base[j][0]
        step_data2 = dict([(key, list(step_data[key])) for key in crv])
        cum_data2 = dict([(key, list(cum_data[key])) for key in crv])
        fwd_data2 = dict([(key, list(fwd_data[key])) for key in crv])

        x_lab = [( str(grps[i]), j[:-3].ljust(10)+' '+month[j][0][i] ) for i in np.arange(len(grps))  for j in crv]
        counts = sum(zip(*step_data2.values()),())  # like an hstack
        color_schem = n_grps*['cornflowerblue','lightcoral','darkseagreen', 'khaki'][:n_crv]
        legend_names = n_grps*[crv[i][:-3] for i in np.arange(n_crv)]

        s1_source = ColumnDataSource(data=dict(x=(x_lab), counts=counts, colors=color_schem, legend_name=legend_names))
        s1 = figure(x_range=FactorRange(*x_lab), width=800, height=500,tools=["pan", "wheel_zoom", "box_zoom", "save", "reset", "help"], toolbar_location='right')
        s1.xgrid.visible = False
        s1.ygrid.visible = False
        s1.vbar(x='x', top='counts', width=0.7, source=s1_source, fill_color='colors', line_color='colors', legend_field='legend_name')
        labels_1 = LabelSet(x='x', y='counts', text='counts', level='glyph', text_align='center', y_offset=-14, source=s1_source, text_font_size='10px', text_color='midnightblue')
        s1.add_layout((labels_1))
        zero_line = Span(location=0, dimension='width', line_color='darkseagreen', line_width=1)
        s1.renderers.extend([zero_line])

        s1.legend.location = 'bottom_right'
        s1.legend.label_text_font = "calibri"
        s1.legend.label_text_font_size = "9pt"
        s1.legend.spacing = 1
        s1.legend.click_policy = "mute"
        s1.xaxis.major_label_orientation = math.pi / 2
        s1.xaxis.axis_label_text_font_size = "2pt"

        cum_counts = sum(zip(*cum_data2.values()),())# like an hstack
        fwd_counts = sum(zip(*fwd_data2.values()), ())
#        color_schem = n_grps*['cornflowerblue','lightcoral','darkseagreen', 'khaki'][:n_crv]
#        legend_names = n_grps*[crv[i][:-3] for i in np.arange(n_crv)]
        x_lab2 = [(str(grps[i]), j[:-3].ljust(10) + str(fwd_base[j][0][i]).rjust(5) + month[j][0][i].rjust(17-len(month[j][0][i]))) for i in np.arange(len(grps)) for j in crv]
        s2_source = ColumnDataSource(data=dict(x=(x_lab2), counts=cum_counts, colors=color_schem, legend_name=legend_names))
        s2 = figure(x_range=FactorRange(*x_lab2), width=800, height=500, tools=["pan", "wheel_zoom", "box_zoom", "save", "reset", "help"], toolbar_location='right')
        s2.xgrid.visible = False
        s2.ygrid.visible = False
        s2.vbar(x='x', top='counts', width=0.7, source=s2_source, fill_color='colors', line_color='colors', legend_field='legend_name')
        labels_2 = LabelSet(x='x', y='counts', text='counts', level='glyph', text_align='center', y_offset=-14, x_offset=-3, source=s2_source, text_font_size='10px', text_color='midnightblue')
        s2.add_layout((labels_2))
        s2.renderers.extend([zero_line])

        s2.y_range = Range1d(min(cum_counts) - 15, max(cum_counts) + 15)
        s2.legend.location = 'bottom_left'
        s2.legend.label_text_font = "calibri"
        s2.legend.label_text_font_size = "9pt"
        s2.legend.spacing = 1
        s2.legend.click_policy = "mute"
        s2.xaxis.major_label_orientation = math.pi / 2
        s2.xaxis.axis_label_text_font_size = "2pt"

        tab1 = TabPanel(child=s1, title="step")
        tab2 = TabPanel(child=s2, title="cum")
        tabs = Tabs(tabs=[tab1, tab2])

    show(tabs)

    return [tabs]



In [16]:
plot_wirp(dt, chg=1)

in
*******_ month ['1', '2', '3', 'Mar-24', 'May-24', 'Jun-24', 'Jul-24', 'Sep-24', 'Nov-24', 'Dec-24', 'Jan-25', 'Mar-25', 'May-25', 'Jun-25', 'Aug-25', 'Oct-25', 'Nov-25', 'Jan-26']
*******_ step [0, 0, 0, -1.0, -16.0, -21.0, -20.0, -23.0, -16.0, -16.0, -17.0, -13.0, -15.0, -16.0, 3.0, -14.0, -6.0, -1.0]
*******_ step_chg [-3.0, -13.0, -16.0, 2.0, -8.0, -12.0, -11.0, -11.0, -4.0, -6.0, -6.0, -2.0, -4.0, -13.0, 12.0, 0, 0, 0]
out


[Tabs(id='p1407', ...)]

In [2]:
import numpy as np
import pandas as pd

from bokeh.models import ColumnDataSource, TapTool
from bokeh.events import Tap

from bokeh.plotting import figure
from bokeh.layouts import column

from bokeh.io import curdoc
from bokeh.client import push_session, pull_session

x = np.random.randn(1001)
y = np.random.randn(1001)

cat1 = np.random.rand(1001) * 10.0
cat2 = np.random.rand(1001) * 10.0

data = np.column_stack((x,y,cat1,cat2))

df = pd.DataFrame(data=data, columns=('x','y','cat1','cat2'))
SRC = ColumnDataSource(df)


TOOLTIPS = [
    ("x", "@x"),
    ("y", "@y"),
    ("Category 1", "@cat1"),
    ("Category 2", "@cat2")]

TOOLS="pan,wheel_zoom,zoom_in,zoom_out,box_zoom,reset,tap"
cplot = figure(tools = TOOLS, tooltips=TOOLTIPS)
cplot.circle("x", "y", source=SRC)

bSource = ColumnDataSource(dict(x=['cat1','cat2'], top=[None]*2))
bplot = figure(x_range=('cat1','cat2'))
bplot.vbar(x='x', top='top', source=bSource)

def callback(event):
    SELECTED = SRC.selected.indices
    print("SELECTED {:}".format(SELECTED))
    if len(SELECTED) > 1:
        SELECTED = SELECTED[-1] # last point in case of multiselect
    bSource.patch(dict(top=[(slice(2),[SRC.data['cat1'][SELECTED],SRC.data['cat2'][SELECTED]])]))

taptool = cplot.select(type=TapTool)
cplot.on_event(Tap, callback)

curdoc().add_root(column(cplot, bplot))



In [3]:
session = push_session(curdoc())
session.show()
session.loop_until_closed()

RuntimeError: Cannot run the event loop while another loop is running

In [ ]:
def plot_opt_vol_surf(v, opt_type = 'P', x_ax = 'delta' ):
    mpl.rcParams.update(mpl.rcParamsDefault)
    fig = plt.figure(figsize=[8,6])
    ax1 = fig.add_subplot(111)
    ax1.set_ylabel('bp vol')
    ax1.set_xlabel(x_ax)
    ax1.set_title(v[0].ticker[:4]+' '+opt_type)

    if opt_type == 'P':
        opt_type_conv = -1
    else:
        opt_type_conv = 1

    for s in v:
        df = s.tab
        df1 = df[df['opt_type']==opt_type_conv]
        df2 = df1[df1['strikes'] == s.center][['strikes','iv','delta']]
        ax1.plot(df1[x_ax],df1['iv'], marker='.',  label = s.ref_date)
        ax1.annotate( str(s.center), xy=(df2[x_ax].tolist()[0], df2['iv'].tolist()[0]), fontsize = 8, horizontalalignment='center', verticalalignment='top')

    ax1.legend(prop={"size":9},  loc='best')
#    plt.tight_layout()
    return fig


In [6]:
#v2 = build_stir_vol_surf(['SFRM4P 96'], chain_len=[20,20], b=0)
#v3 = build_vol_surf(['FVH4P 111'], chain_len=[12,12], b=0)
v2 = build_vol_surf(['RXJ4P 133'], chain_len=[12,12], b=0)

0 RXJ4P 127.5 Comdty
1 RXJ4P 128.0 Comdty
2 RXJ4P 128.5 Comdty
3 RXJ4P 129.0 Comdty
4 RXJ4P 129.5 Comdty
5 RXJ4P 130.0 Comdty
6 RXJ4P 130.5 Comdty
7 RXJ4P 131.0 Comdty
8 RXJ4P 131.5 Comdty
9 RXJ4P 132.0 Comdty
10 RXJ4P 132.5 Comdty
11 RXJ4P 133.0 Comdty
12 RXJ4P 133.5 Comdty
13 RXJ4P 134.0 Comdty
14 RXJ4P 134.5 Comdty
15 RXJ4P 135.0 Comdty
16 RXJ4P 135.5 Comdty
17 RXJ4P 136.0 Comdty
18 RXJ4C 131.0 Comdty
19 RXJ4C 131.5 Comdty
20 RXJ4C 132.0 Comdty
21 RXJ4C 132.5 Comdty
22 RXJ4C 133.0 Comdty
23 RXJ4C 133.5 Comdty
24 RXJ4C 134.0 Comdty
25 RXJ4C 134.5 Comdty
26 RXJ4C 135.0 Comdty
27 RXJ4C 135.5 Comdty
28 RXJ4C 136.0 Comdty
29 RXJ4C 136.5 Comdty
30 RXJ4C 137.0 Comdty
31 RXJ4C 137.5 Comdty
32 RXJ4C 138.0 Comdty
33 RXJ4C 138.5 Comdty
34 RXJ4C 139.0 Comdty
35 RXJ4C 139.5 Comdty
36 RXJ4C 140.0 Comdty
*** !!! listed_vol surf built !!! ***


In [7]:
v2.tab

,ticker,strikes,opt_type,px,px_64,Yld,ATM_K,bs_px,iv,delta,gamma,theta,vega
0,RXJ4P 127.5 Comdty,127.5,-1,0.05,0'03,2.7897898438141993,59.45445033396486,0.050001386885864446,8.246907019583748,-3.6018433799028164,0.022858420544551762,-1.384021744948639,3.3151612005415423
1,RXJ4P 128.0 Comdty,128.0,-1,0.065,0'04,2.7392410843874417,54.39957439128911,0.06500039632354883,8.067762233172093,-4.610701452063795,0.028547464974346053,-1.6539959466758074,4.050307289799925
2,RXJ4P 128.5 Comdty,128.5,-1,0.08499999999999999,0'05,2.6889248734194986,49.3679532944948,0.08503539344594736,7.900001798249148,-5.914670043201423,0.03552612063345331,-1.9733246845749426,4.935626764660327
3,RXJ4P 129.0 Comdty,129.0,-1,0.11,0'07,2.638839207913102,44.35938674385511,0.11001423590118359,7.714711350490612,-7.512096680659347,0.04376821153102787,-2.318037681550001,5.938076884298909
4,RXJ4P 129.5 Comdty,129.5,-1,0.14500000000000002,0'09,2.5889821100541175,39.37367695795668,0.14515560906405983,7.570548124670186,-9.62871205692037,0.05371298692495643,-2.7388129795133436,7.151118299948438
5,RXJ4P 130.0 Comdty,130.0,-1,0.195,0'12,2.53935162679633,34.41062863217792,0.19492079832104298,7.477237709116535,-12.40456106966079,0.06522330503675418,-3.2433460296281487,8.576524325258955
6,RXJ4P 130.5 Comdty,130.5,-1,0.255,0'16,2.48994582945466,29.470048898010948,0.2549752993126219,7.343707241993763,-15.642299051838393,0.07774049675559784,-3.7277389653334025,10.039914192073695
7,RXJ4P 131.0 Comdty,131.0,-1,0.33999999999999997,0'21,2.440762813306624,24.551747283207348,0.3402282011465978,7.288333977838231,-19.769033339977966,0.09082020734105523,-4.28766673359136,11.640672977607828
8,RXJ4P 131.5 Comdty,131.5,-1,0.445,0'28,2.3918006972018584,19.65553567273077,0.4448510394145305,7.214493485406372,-24.49508264931744,0.10371038863746686,-4.795001178898218,13.158168441144342
9,RXJ4P 132.0 Comdty,132.0,-1,0.58,0'37,2.3430576231795244,14.78122827049737,0.5799428463425398,7.181554853657418,-29.959377708786572,0.11513546549889951,-5.271194716687895,14.541022201004902


In [15]:
v3.tab

,ticker,strikes,opt_type,px,px_64,Yld,ATM_K,bs_px,iv,delta,gamma,theta,vega
0,FVH4P 103.5 Comdty,103.5,-1,0.0078125,0'00,5.0952958805907205,81.1057792032849,0.007815612779134437,9.032313528372638,-1.4541649419239235,0.02435112067223368,-1.1313575589247957,0.6179307387023195
1,FVH4P 103.75 Comdty,103.75,-1,0.0078125,0'00,5.031705156139907,74.74670675820354,0.007813786455786726,8.420394387381409,-1.548489405446285,0.02757050274383166,-1.113242862778134,0.6522272819716894
2,FVH4P 104.0 Comdty,104.0,-1,0.0078125,0'00,4.968293501410954,68.40554128530823,0.007813099117971359,7.804642156914303,-1.6575406919642928,0.031532304875717225,-1.0938032626695087,0.6914018778629654
3,FVH4P 104.25 Comdty,104.25,-1,0.0078125,0'00,4.905059970861826,62.08218823039546,0.007826521548126329,7.18615977246604,-1.7874547379127537,0.0365265970588038,-1.074180246960405,0.7374420955832744
4,FVH4P 104.5 Comdty,104.5,-1,0.0078125,0'00,4.842003626252476,55.77655376946042,0.0077991721721465,6.556839780171135,-1.933934654647564,0.042808613016731924,-1.0480715406210899,0.7885832915288319
5,FVH4P 104.75 Comdty,104.75,-1,0.0078125,0'00,4.779123536570772,49.48854480129006,0.0078123975865380104,5.926383803928358,-2.1194452633803618,0.05118706759273415,-1.0237786249050451,0.8522595305977394
6,FVH4P 105.0 Comdty,105.0,-1,0.015625,0'01,4.7164187779593245,43.2180689401453,0.01562524143557749,5.956425096425526,-3.8807946492207464,0.08419438287252168,-1.700923943858895,1.4089339857815986
7,FVH4P 105.25 Comdty,105.25,-1,0.01953125,0'01,4.6538884336432496,36.96503450853781,0.01953374195121336,5.485518256111755,-5.0574870981338735,0.11316742149177131,-1.9389503486603503,1.7440580165378345
8,FVH4P 105.5 Comdty,105.5,-1,0.03515625,0'02,4.591531593858778,30.72935053009065,0.035159523327610694,5.429652235712191,-8.411844329936073,0.16959499397668376,-2.8465217618652097,2.5870622894983715
9,FVH4P 105.75 Comdty,105.75,-1,0.05078125,0'03,4.52934735578292,24.510926722504855,0.05078227268436603,5.09108949479754,-12.041134716126782,0.23489577156225502,-3.4657808174343767,3.35975626746475


In [7]:
vol_surf = [v2]

df_call = [vol_surf[i].tab[vol_surf[i].tab['opt_type'] == 1] for i in np.arange(len(vol_surf))]
df_put = [vol_surf[i].tab[vol_surf[i].tab['opt_type'] == -1] for i in np.arange(len(vol_surf))]

In [8]:

s1 = figure(width=550, height=400,tools=["pan", "crosshair", "wheel_zoom", "box_zoom", "save", "reset", "help"], toolbar_location='right')
s1.add_tools(HoverTool(tooltips=[('x', '$x'), ('y', '$y')]))
s1.xgrid.visible = False
s1.ygrid.visible = False
for i in np.arange(len(df_call)):
    s1.line(x=df_call[i]['delta'] , y=df_call[i]['iv'] , width=0.7, color='royalblue', legend_label="opt_test")
    s1.scatter(x=df_call[i]['delta'] , y=df_call[i]['iv'] , width=0.7, color='navy', size=10, marker="dot")
    live_line = Span(location=50, dimension='height', line_color='goldenrod', line_width=1)
    s1.add_layout(live_line)
s1.legend.location = 'bottom_right'
s1.legend.label_text_font = "calibri"
s1.legend.label_text_font_size = "9pt"
s1.legend.spacing = 1
s1.legend.click_policy = "mute"
s1.legend.background_fill_alpha = 0.0
s1.yaxis.axis_label = 'imp. vol.'
s1.xaxis.axis_label = 'delta'

s2 = figure(width=550, height=400,tools=["pan", "crosshair", "wheel_zoom", "box_zoom", "save", "reset", "help"], toolbar_location='right')
s2.add_tools(HoverTool(tooltips=[('x', '$x'), ('y', '$y')]))
s2.xgrid.visible = False
s2.ygrid.visible = False
for i in np.arange(len(df_call)):
    s2.line(x=df_call[i]['strikes'] , y=df_call[i]['iv'] , width=0.7, color='royalblue', legend_label="opt_test")
    s2.scatter(x=df_call[i]['strikes'] , y=df_call[i]['iv'] , width=0.7, color='navy', size=10, marker="dot")
    live_line = Span(location=110, dimension='height', line_color='goldenrod', line_width=1)
    s2.add_layout(live_line)
s2.legend.location = 'bottom_right'
s2.legend.label_text_font = "calibri"
s2.legend.label_text_font_size = "9pt"
s2.legend.spacing = 1
s2.legend.click_policy = "mute"
s2.legend.background_fill_alpha = 0.0
s2.yaxis.axis_label = 'imp. vol.'
s2.xaxis.axis_label = 'K'

s3 = figure(width=550, height=400,tools=["pan", "crosshair", "wheel_zoom", "box_zoom", "save", "reset", "help"], toolbar_location='right')
s3.add_tools(HoverTool(tooltips=[('x', '$x'), ('y', '$y')]))
s3.xgrid.visible = False
s3.ygrid.visible = False
for i in np.arange(len(df_put)):
    s3.line(x=df_put[i]['delta'] , y=df_put[i]['iv'] , width=0.7, color='palevioletred', legend_label="opt_test")
    s3.scatter(x=df_put[i]['delta'] , y=df_put[i]['iv'] , width=0.7, color='red', size=10, marker="dot")
    live_line = Span(location=-50, dimension='height', line_color='goldenrod', line_width=1)
    s3.add_layout(live_line)
s3.legend.location = 'bottom_right'
s3.legend.label_text_font = "calibri"
s3.legend.label_text_font_size = "9pt"
s3.legend.spacing = 1
s3.legend.click_policy = "mute"
s3.legend.background_fill_alpha = 0.0
s3.yaxis.axis_label = 'imp. vol.'
s3.xaxis.axis_label = 'delta'

s4 = figure(width=550, height=400,tools=["pan", "crosshair", "wheel_zoom", "box_zoom", "save", "reset", "help"], toolbar_location='right')
s4.add_tools(HoverTool(tooltips=[('x', '$x'), ('y', '$y')]))
s4.xgrid.visible = False
s4.ygrid.visible = False
for i in np.arange(len(df_put)):
    s4.line(x=df_put[i]['strikes'] , y=df_put[i]['iv'] , width=0.7, color='palevioletred', legend_label="opt_test")
    s4.scatter(x=df_put[i]['strikes'] , y=df_put[i]['iv'] , width=0.7, color='red', size=10, marker="dot")
    live_line = Span(location=110, dimension='height', line_color='goldenrod', line_width=1)
    s4.add_layout(live_line)
s4.legend.location = 'bottom_right'
s4.legend.label_text_font = "calibri"
s4.legend.label_text_font_size = "9pt"
s4.legend.spacing = 1
s4.legend.click_policy = "mute"
s4.legend.background_fill_alpha = 0.0
s4.yaxis.axis_label = 'imp. vol.'
s4.xaxis.axis_label = 'K'


tab1 = TabPanel(child=s2, title="call strikes")
tab2 = TabPanel(child=s1, title="call delta")
tab3 = TabPanel(child=s3, title="put delta")
tab4 = TabPanel(child=s4, title="put strikes")

tabs = Tabs(tabs=[tab1, tab2, tab3, tab4])

show(tabs)



In [9]:
df_call[0]

,ticker,strikes,opt_type,px,px_64,Yld,ATM_K,bs_px,iv,delta,gamma,theta,vega


In [ ]:
def plt_opt_strat(st, add_delta = [0] , payoff_increm_calc = 100, strat_pv_increm = 0.25):
    mpl.rcParams.update(mpl.rcParamsDefault)
#    st = bond_fut_opt_strat(['RXH2'],['P','P'],[170.5,169.5],[1,-1], s_range = [-1,1], increm = 0.5, chain_len = [6,6])
#    add_delta = [0]

    spot_idx = st.strat.loc[st.strat['ATM_K'] == 0].index[0]
    if st.currency == 'USD':
        px_label = px_dec_to_opt_frac(st.strat_px)
    else:
        px_label = str(np.round(st.strat_px,3))
#    plt.rcParams['axes.labelpad'] = 4.00
    fig = plt.figure(figsize=[8,6])
    #fig.subplots_adjust(top=0.8)
    ax1 = fig.add_subplot(211)
    ax1.set_ylabel('strat_px')
    ax1.set_title(st.strat_name+'       px:'+px_label+'       delta:'+str(np.round(st.strat['strat_delta'][spot_idx],1)))

    x_spot = st.strat['fut_px'][spot_idx]
    #### payoff at expiry
    x1 = np.linspace(min(st.strat['fut_px']),max(st.strat['fut_px']), num = payoff_increm_calc)
    if len(add_delta) > 1:
        d_payoff1 = fut_payoff(x1, [st.fut], add_delta)
        y1 = opt_payoff(x1, st.opt_dets, st.strat_px) + d_payoff1
    else:
        y1 = opt_payoff(x1, st.opt_dets, st.strat_px)

    #### strategy px
    x2 = st.strat['fut_px']
    y2 = st.strat['strat_px']

    sc1 = ax1.plot(x1, y1, lw=0.5, color = 'k')
    sc2 = ax1.plot(x2, y2, marker = 'x', markersize = 5, lw=0.5)
    sc_spot = ax1.axvline(x_spot, lw=0.5, color = 'y')
    sc_be = ax1.axhline(0, lw=0.5, color = 'k')

    ax1.tick_params(axis='both', which='major', labelsize=9)
    new_labels = [str(st.strat['fut_px'][i])+'\n\n'+str(np.round(st.strat['ATM_K'][i],1))+'\n\n'+str(np.round(st.strat['strat_delta'][i],1)) for i in np.arange(len(st.strat))]
    new_labels.pop(spot_idx )
    ax1.set_xticks(x2[x2.index != spot_idx].tolist())
    ax1.set_xticklabels(new_labels)

    payoff_ticks = np.arange(round_nearest(min(min(y2),min(y1)),strat_pv_increm), round_nearest(max(max(y2),max(y1))+strat_pv_increm,strat_pv_increm), strat_pv_increm)
    ax1.set_yticks(payoff_ticks)
    if st.currency == 'USD':
        ax1.set_yticklabels([px_dec_to_opt_frac(payoff_ticks[i]) for i in np.arange(len(payoff_ticks))])
    else:
        ax1.set_yticklabels([payoff_ticks[i] for i in np.arange(len(payoff_ticks))])


    ### delta sub_plot
    ax2 = fig.add_subplot(212)
    ax2.set_ylabel('strat_delta')
    plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.25)

    x3 = st.strat['fut_px']
    y3 = st.strat['strat_delta']/100

    sc3 = ax2.plot(x3, y3, marker = 'x', markersize = 5, lw=0.5, color = 'g')
    sc_spot_2 = ax2.axvline(x_spot, lw=0.5, color = 'y')

    if len(add_delta) > 1:
        sc3 = ax2.plot(x3, y3+add_delta[0] , marker = 'x', markersize = 5, lw=0.5, color = 'k', ls ='dashed')
        st.strat['strat_delta_updt'] = y3+add_delta[0]

    #plt.show()
    plt.tight_layout()
    return fig


In [121]:
v1 = build_vol_surf(['FVH4P 107'], chain_len=[15,15], b=0)


*** !!! listed_vol surf built !!! ***


In [9]:
st = bond_fut_opt_strat(['RXJ4'],['C','C'],[135,136.5],[1,-1], s_range = [-1,1], increm = 0.5, built_surf=v2)

In [10]:
[float(st.strat['strat_px_fmt'][i]) for i in np.arange(len(st.strat))]

ValueError: could not convert string to float: '1-05.2'

In [11]:
[st.strat['strat_px_fmt'][i].split('-') for i in np.arange(len(st.strat))]

[['12.5'],
 ['16.0'],
 ['20.2'],
 ['20.5'],
 ['25.3'],
 ['30.8'],
 ['37.1'],
 ['43.8'],
 ['50.4'],
 ['57.0'],
 ['63.2'],
 ['1', '05.2']]

In [133]:
st.strat['strat_px_fmt']

0     -1-16.8
1     -1-01.6
2       -50.6
3       -36.1
4       -22.1
5        -9.7
6        -0.1
7         6.5
8        10.2
9        11.4
10       10.5
11        8.9
12        6.5
13        6.4
14        4.9
15        2.7
16        1.5
17        0.2
Name: strat_px_fmt, dtype: object

In [12]:
add_delta = [0]
payoff_increm_calc = 100    ### stick diagram increment
strat_pv_increm = 0.25


spot_idx = st.strat.loc[st.strat['ATM_K'] == 0].index[0]
if st.currency == 'USD':
    px_label = px_dec_to_opt_frac(st.strat_px)
else:
    px_label = str(np.round(st.strat_px,3))

x_spot = st.strat['fut_px'][spot_idx]
#### payoff at expiry
x1 = np.linspace(min(st.strat['fut_px']),max(st.strat['fut_px']), num = payoff_increm_calc)
if len(add_delta) > 1:
    d_payoff1 = fut_payoff(x1, [st.fut], add_delta)
    y1 = opt_payoff(x1, st.opt_dets, st.strat_px) + d_payoff1
    y1_fmt = [ float(px_dec_to_opt_frac(y1[i])) for i in np.arange(len(y1))]
else:
    y1 = opt_payoff(x1, st.opt_dets, st.strat_px)
    y1_fmt = [ float(px_dec_to_opt_frac(y1[i])) for i in np.arange(len(y1))]



ValueError: could not convert string to float: '1-01.2'

In [119]:
s1 = figure(width=700, height=350,tools=["pan", "crosshair", "wheel_zoom", "box_zoom", "save", "reset", "help"], toolbar_location='right')
main_source = ColumnDataSource(data={'x2':st.strat['fut_px'], 'y2':st.strat['strat_px'], 'k_dist':st.strat['ATM_K'], 'y2_fmt': [float(st.strat['strat_px_fmt'][i]) for i in np.arange(len(st.strat))]} )
s1.add_tools(HoverTool(tooltips=[('fut', '$x{0.00}'), ('px', '@y2{0.00}'), ('px_fmt', '$y{0.0}')]))
s1.xgrid.visible = False
s1.ygrid.visible = False
if st.currency == 'USD':
    s1.line(x=x1, y=y1_fmt , width=0.7, color='black', alpha=1.0, muted_alpha=0.1, legend_label="@expiry")
    s1.line(x='x2', y='y2_fmt' , width=0.7, color='navy', alpha=1.0, muted_alpha=0.1, legend_label="opt_px: "+str(px_dec_to_opt_frac(st.strat_px)), source = main_source)
    s1.yaxis.axis_label = 'strat_px (64th)'
else:
    s1.line(x=x1, y=y1 , width=0.7, color='black', alpha=1.0, muted_alpha=0.1, legend_label="@expiry")
    s1.line(x='x2', y='y2' , width=0.7, color='navy', alpha=1.0, muted_alpha=0.1, legend_label="opt_px: "+str(st.strat_px), source = main_source)
    s1.yaxis.axis_label = 'strat_px'
live_line = Span(location=x_spot, dimension='height', line_color='goldenrod', line_width=1)
zero_line = Span(location=0, dimension='width', line_color='darkseagreen', line_width=1)
s1.add_layout(live_line)
s1.add_layout(zero_line)
s1_source = ColumnDataSource(data=dict(x= np.round(st.strat['fut_px'][::int(len(st.strat)/6)],1).tolist(), z= np.round(st.strat['ATM_K'][::int(len(st.strat)/6)],1).tolist()))
labels_yld = LabelSet(x='x', y=5, y_units='screen', text='z', source=s1_source, text_font='calibri', text_color='firebrick', text_font_size='9px', text_align='center', x_offset=0)
s1.add_layout(labels_yld)
s1.xaxis.ticker = np.round(st.strat['fut_px'][::int(len(st.strat)/6)],2)
s1.legend.location = 'top_left'
s1.legend.label_text_font = "calibri"
s1.legend.label_text_font_size = "9pt"
s1.legend.spacing = 1
s1.legend.click_policy = "mute"
s1.legend.background_fill_alpha = 0.0

s2_source = ColumnDataSource(data=dict(x= np.round(st.strat['fut_px'][::int(len(st.strat)/6)],1).tolist(), z= np.round(0.01*st.strat['strat_delta'][::int(len(st.strat)/6)],2).tolist()))
s2 = figure(width=700, height=275,tools=["pan", "crosshair", "wheel_zoom", "box_zoom", "save", "reset", "help"], toolbar_location='right')
s2.add_tools(HoverTool(tooltips=[('x{0.00}', '$x'), ('y', '$y{0.00}')]))
s2.xgrid.visible = False
s2.ygrid.visible = False
s2.line(x=st.strat['fut_px'] , y=st.strat['strat_delta']/100 , width=0.7, color='green', alpha=1.0, muted_alpha=0.1, legend_label="delta")
live_line = Span(location=x_spot, dimension='height', line_color='goldenrod', line_width=1)
labels_delta = LabelSet(x='x', y=5, y_units='screen', text='z', source=s2_source, text_font='calibri', text_color='firebrick', text_font_size='9px', text_align='center', x_offset=0)
s2.xaxis.ticker = np.round(st.strat['fut_px'][::int(len(st.strat)/6)],2)
s2.add_layout(live_line)
s2.add_layout(labels_delta)
s2.legend.location = 'top_left'
s2.legend.label_text_font = "calibri"
s2.legend.label_text_font_size = "9pt"
s2.legend.spacing = 1
s2.legend.click_policy = "mute"
s2.legend.background_fill_alpha = 0.0
s2.yaxis.axis_label = 'strat_delta'

grid = gridplot([[s1],[s2]])

show(grid)



In [120]:
st.fut

'TYH4 Comdty'

In [116]:
st.strat['strat_px']

0    0.025886545195285944
1     0.06741572261384512
2     0.10168093105603315
3      0.1403577022358249
4      0.1832162291957082
5     0.19514433336346385
6     0.22284995299706178
7      0.2615440850619258
8      0.2785514314045184
9     0.26914799073914997
10    0.21961467580622762
11    0.14450287120867777
12    0.04382004322900612
13   -0.08827897515301752
14    -0.2502951638508377
Name: strat_px, dtype: float64